# Part 1: Description

Melbourne, where I am currently staying, is the capital and most populous city of the Australian state of Victoria, and the second most populous city in Australia and Oceania. It has a population of 5 million (19% of the population of Australia). At the city's centre is the modern Federation Square development, with plazas, bars, and restaurants by the Yarra River. So, it is definitely one of the best places to start up a new business in Australia.

<img src="Melbourne.jpg" width = 350>

During the daytime, specially in the morning and lunch hours, office areas provide huge opportunities for restaurants. On the other hand, sushi is perhaps the best known Japanese food dish to the outside world. It is a recipe that consists of rice and fish with a flavor of vinegar. This food is one of the best foods among Australians and there are many sushi restaurants around the Australia, and particularly Victoria state.

<img src="sushi.jpg" width = 250>

#### During this project, I have focued on opening a new sushi restaurant as staring a new business in Victoria states 

I have focused on different neighbourhoods in Victoria state and compared them together. First I have obtained the Victoria state neighbourhood names, population and size. Then, their latitude, longitude, and their density is obtained using foursquare API.

We have then, focused on 4 neighbourhoods with highest density, and compared together using the number of existing sushi restaurant in each of them. That information is achinevd using foursqaure API.

After getting number of sushi restaurant in each neighbourhood, the one with the least is selected as the potential place for opening the new sushi restauarnt. 

#### Now, We will go through each step of this project and address them separately. I first outline the initial data preparation and describe future steps to start the battle of neighborhoods in Victoria state, Australia.

# Part 2: implementation

In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [8]:
df = pd.read_html('https://en.wikipedia.org/wiki/Local_government_areas_of_Victoria', flavor='bs4')

In [9]:
df1 = df[0]

In [10]:
df1.columns

MultiIndex(levels=[['Council seat', 'Councillors(2012)', 'Land area', 'Local government area', 'Map', 'Notes', 'Population(2001 census)', 'Population(2016 census)', 'Region', 'Year est.'], ['Council seat', 'Councillors(2012)', 'Density(2016)', 'Local government area', 'Map', 'Notes', 'Population(2001 census)', 'Population(2016 census)', 'Region', 'Year est.', 'km²', 'sq mi']],
           codes=[[3, 0, 8, 9, 2, 2, 2, 6, 7, 1, 5, 4], [3, 0, 8, 9, 10, 11, 2, 6, 7, 1, 5, 4]])

In [12]:
df1.head()

Local government area Council seat                  Region  \
  Local government area Council seat                  Region   
0     City of Melbourne    Melbourne         Inner Melbourne   
1  City of Port Phillip     St Kilda         Inner Melbourne   
2   City of Stonnington      Malvern         Inner Melbourne   
3         City of Yarra     Richmond         Inner Melbourne   
4       City of Banyule      Ivanhoe  Metropolitan Melbourne   

          Year est. Land area                     Population(2001 census)  \
          Year est.       km² sq mi Density(2016) Population(2001 census)   
0    12 August 1842        37    14          3486               60,745[1]   
1      22 June 1994        21     8          5101               78,227[3]   
2      22 June 1994        26    10          4152               86,090[5]   
3      22 June 1994        20     8          4458               67,052[7]   
4  15 December 1994        63    24          2004              113,696[9]   

  Population(2016 census) Councillors(2012)  \
  Population(2016 census) Councillors(2012)   
0              135,959[2]                11   
1              100,863[4]                 7   
2              103,832[6]                 9   
3               86,657[8]                 9   
4             121,865[10]                 7   

                                               Notes Map  
                                               Notes Map  
0                          Capital city of Victoria. NaN  
1  Amalgamation of the cities of St Kilda, South ... NaN  
2  Amalgamation of the cities of Malvern and Prahran NaN  
3  Amalgamation of the cities of Richmond, Collin... NaN  
4  Amalgamation of the City of Heidelberg and par... NaN

In [13]:
df_Melbourne = df1[['Local government area', 'Land area', 'Population(2016 census)']]

In [14]:
df_Melbourne.head()

Local government area Land area                     Population(2016 census)
  Local government area       km² sq mi Density(2016) Population(2016 census)
0     City of Melbourne        37    14          3486              135,959[2]
1  City of Port Phillip        21     8          5101              100,863[4]
2   City of Stonnington        26    10          4152              103,832[6]
3         City of Yarra        20     8          4458               86,657[8]
4       City of Banyule        63    24          2004             121,865[10]

In [15]:
df_Melbourne.to_csv('Melbourne1.csv')

### Reading data of Victoria state in Australia

In [16]:
df_Melbourne = pd.read_csv('Melbourne.csv')

In [17]:
df_Melbourne.drop('Unnamed: 0', axis=1, inplace=True)

In [18]:
df_Melbourne.head()

,Area,Area_SqKm,Population
0,City of Melbourne,14,135959
1,City of Port Phillip,8,100863
2,City of Stonnington,10,103832
3,City of Yarra,8,86657
4,City of Banyule,24,121865


In [19]:
CLIENT_ID = 'R4OQ3SNGYWRVB3YMRLRZPWBNSBRHBEKL3QZ3HJU4DXHPXX3J' # your Foursquare ID
CLIENT_SECRET = 'MXMH14IELSJAXDRUEMABGHLYGRUEW0TH52FX4SYVE1LDJ2M1' # your Foursquare Secret
VERSION = '20190805'
LIMIT = 300
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: R4OQ3SNGYWRVB3YMRLRZPWBNSBRHBEKL3QZ3HJU4DXHPXX3J
CLIENT_SECRET:MXMH14IELSJAXDRUEMABGHLYGRUEW0TH52FX4SYVE1LDJ2M1


### Visualising the melbourne city

In [20]:
address = 'Melbourne'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address, timeout=20)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-37.8142176 144.9631608


In [21]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Melbourne',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# display map
venues_map

### Here we obtain the latitude and longitude of all victoria states

In [22]:
data = []
for address in df_Melbourne['Area']:
  #  address = 'Melbourne'

    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address, timeout=200)
    latitude = location.latitude
    longitude = location.longitude
 #   print(latitude, longitude)
    
    data.append((latitude, longitude))
    
cols=['latitude','longitude']
df_ll = pd.DataFrame(data, columns=cols)

In [23]:
df_Melbourne['latitude'] = df_ll['latitude']
df_Melbourne['longitude'] = df_ll['longitude']
df_Melbourne.head()

,Area,Area_SqKm,Population,latitude,longitude
0,City of Melbourne,14,135959,-37.814218,144.963161
1,City of Port Phillip,8,100863,-37.858645,144.982941
2,City of Stonnington,10,103832,-37.861385,145.039141
3,City of Yarra,8,86657,-37.805070,144.986540
4,City of Banyule,24,121865,-37.733876,145.086813


Now the data of melbourne has the name of area, the population, the size, and corresponding latitude and langitute

### Another important point that we here calculate is the density of each area with devising population of each area by size of each area

In [24]:
df_Melbourne['Ratio'] = df_Melbourne['Population'] / df_Melbourne['Area_SqKm']

In [25]:
df_Melbourne.head()

,Area,Area_SqKm,Population,latitude,longitude,Ratio
0,City of Melbourne,14,135959,-37.814218,144.963161,9711.357143
1,City of Port Phillip,8,100863,-37.858645,144.982941,12607.875000
2,City of Stonnington,10,103832,-37.861385,145.039141,10383.200000
3,City of Yarra,8,86657,-37.805070,144.986540,10832.125000
4,City of Banyule,24,121865,-37.733876,145.086813,5077.708333


In [26]:
df_Melbourne.sort_values('Ratio', ascending=False, inplace=True)
df_Melbourne.reset_index()
df_Melbourne.head(4)

,Area,Area_SqKm,Population,latitude,longitude,Ratio
1,City of Port Phillip,8,100863,-37.858645,144.982941,12607.875000
3,City of Yarra,8,86657,-37.805070,144.986540,10832.125000
2,City of Stonnington,10,103832,-37.861385,145.039141,10383.200000
0,City of Melbourne,14,135959,-37.814218,144.963161,9711.357143


#### Now the data is ready 

### Here we consider the sushi restaurant as our target venue to be examined

In [27]:
search_query = 'sushi'
radius = 2000
print(search_query + ' .... OK!')

sushi .... OK!


### Here we are considering four area with highest density portion and comparing those areas to select the place with lowest number of sushi restaurant

### Place 1: City of Port Phillip

In [28]:
latitude  = df_Melbourne.iloc[0,3]
longitude = df_Melbourne.iloc[0,4]

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=R4OQ3SNGYWRVB3YMRLRZPWBNSBRHBEKL3QZ3HJU4DXHPXX3J&client_secret=MXMH14IELSJAXDRUEMABGHLYGRUEW0TH52FX4SYVE1LDJ2M1&ll=-37.858645,144.982940539032&v=20190805&query=sushi&radius=2000&limit=300'

In [29]:
results = requests.get(url).json()

In [30]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,4b148749f964a52045a423e3,11/1 Commercial Road,AU,St Kilda,Australia,NaN,1552,"[11/1 Commercial Road, St Kilda VIC 3004, Aust...","[{'label': 'display', 'lat': -37.8449659166809...",-37.844966,144.979516,NaN,3004,VIC,Shuji Sushi,v-1565310229
1,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,4b454743f964a520c20926e3,10 Queens Rd,AU,Melbourne,Australia,NaN,2387,"[10 Queens Rd, Melbourne VIC 3000, Australia]","[{'label': 'display', 'lat': -37.8381928612084...",-37.838193,144.974769,NaN,3000,VIC,Tower Sushi,v-1565310229
2,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,5b4064273b4e00002c47f143,151 Chapel St,AU,Windsor,Australia,NaN,1104,"[151 Chapel St, Windsor VIC 3181, Australia]","[{'label': 'display', 'lat': -37.852764, 'lng'...",-37.852764,144.993067,NaN,3181,VIC,Ganbare Kaz Kaiten Sushi,v-1565310229
3,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",False,4cd621bda5b346883f639250,NaN,AU,NaN,Australia,NaN,584,[Australia],"[{'label': 'display', 'lat': -37.860312, 'lng'...",-37.860312,144.976637,NaN,NaN,NaN,Sushi & Grill,v-1565310229
4,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,4c54defba724e21e357810f7,NaN,AU,NaN,Australia,NaN,403,[Australia],"[{'label': 'display', 'lat': -37.8604173195367...",-37.860417,144.978940,NaN,NaN,NaN,St Kilda Sushi & Noodle Bar,v-1565310229


In [31]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Shuji Sushi,Sushi Restaurant,11/1 Commercial Road,AU,St Kilda,Australia,NaN,1552,"[11/1 Commercial Road, St Kilda VIC 3004, Aust...","[{'label': 'display', 'lat': -37.8449659166809...",-37.844966,144.979516,NaN,3004,VIC,4b148749f964a52045a423e3
1,Tower Sushi,Sushi Restaurant,10 Queens Rd,AU,Melbourne,Australia,NaN,2387,"[10 Queens Rd, Melbourne VIC 3000, Australia]","[{'label': 'display', 'lat': -37.8381928612084...",-37.838193,144.974769,NaN,3000,VIC,4b454743f964a520c20926e3
2,Ganbare Kaz Kaiten Sushi,Sushi Restaurant,151 Chapel St,AU,Windsor,Australia,NaN,1104,"[151 Chapel St, Windsor VIC 3181, Australia]","[{'label': 'display', 'lat': -37.852764, 'lng'...",-37.852764,144.993067,NaN,3181,VIC,5b4064273b4e00002c47f143
3,Sushi & Grill,Korean Restaurant,NaN,AU,NaN,Australia,NaN,584,[Australia],"[{'label': 'display', 'lat': -37.860312, 'lng'...",-37.860312,144.976637,NaN,NaN,NaN,4cd621bda5b346883f639250
4,St Kilda Sushi & Noodle Bar,Sushi Restaurant,NaN,AU,NaN,Australia,NaN,403,[Australia],"[{'label': 'display', 'lat': -37.8604173195367...",-37.860417,144.978940,NaN,NaN,NaN,4c54defba724e21e357810f7
5,West Point Sushi Noodles,Asian Restaurant,NaN,AU,Point Cook,Australia,NaN,473,"[Point Cook VIC, Australia]","[{'label': 'display', 'lat': -37.8545001432123...",-37.854500,144.981721,NaN,NaN,VIC,4e5f0b19d22d7239c1ab56ac
6,Sushi San,Sushi Restaurant,NaN,AU,Prahran,Australia,NaN,1258,"[Prahran VIC 3181, Australia]","[{'label': 'display', 'lat': -37.84909, 'lng':...",-37.849090,144.990590,NaN,3181,VIC,5c1471e1d0a149002ceab4dd
7,Sushi Handroll,Sushi Restaurant,145 High St,AU,Prahran,Australia,NaN,1125,"[145 High St, Prahran VIC 3181, Australia]","[{'label': 'display', 'lat': -37.85152, 'lng':...",-37.851520,144.992020,NaN,3181,VIC,4bdf915f6198c9b6f9d117ff
8,Hana Sushi Bar,Sushi Restaurant,618b St Kilda Road,AU,Melbourne,Australia,NaN,685,"[618b St Kilda Road, Melbourne VIC, Australia]","[{'label': 'display', 'lat': -37.85267, 'lng':...",-37.852670,144.981050,NaN,NaN,VIC,4c7343827121a1cd4aec64d1
9,Hako Sushi,Sushi Restaurant,234 Chapel St,AU,Prahran,Australia,NaN,1315,"[234 Chapel St, Prahran VIC, Australia]","[{'label': 'display', 'lat': -37.8503598181980...",-37.850360,144.993619,NaN,NaN,VIC,4b11ea0af964a520e88623e3


In [32]:
dataframe_filtered['name'].count()

34

In [33]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Place 2: City of Yarra

In [34]:
latitude  = df_Melbourne.iloc[1,3]
longitude = df_Melbourne.iloc[1,4]

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered2 = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered2['categories'] = dataframe_filtered2.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered2.columns = [column.split('.')[-1] for column in dataframe_filtered2.columns]

dataframe_filtered2['name'].count()

50

In [42]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered2.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Place 3: City of Stonnington

In [36]:
latitude  = df_Melbourne.iloc[2,3]
longitude = df_Melbourne.iloc[2,4]

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered3 = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered3['categories'] = dataframe_filtered3.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered3.columns = [column.split('.')[-1] for column in dataframe_filtered3.columns]

dataframe_filtered3['name'].count()

8

In [37]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered3.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Place 4: City of Melbourne

In [38]:
latitude  = df_Melbourne.iloc[3,3]
longitude = df_Melbourne.iloc[3,4]

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered4 = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered4['categories'] = dataframe_filtered4.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered4.columns = [column.split('.')[-1] for column in dataframe_filtered4.columns]

dataframe_filtered4['name'].count()

50

In [39]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered4.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### After Studying four areas with highest density, It can be observed that  'City of Stonnington' with only 8 suchi restaurant is the best potential place to open a sushi restaurant

### This place is the third highest populated area in victoria state with population of 103832 but with only 8 sushi restaurants